<a href="https://colab.research.google.com/github/Asif1310/3009/blob/main/Cricketanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.mkdir('/content/json_data') # to create a directory to store all json files

In [2]:
# downloading zip file from the URL and extracting it to the new directory created in previous step
import zipfile 
!wget -O '/content/json_data/main.zip' 'https://cricsheet.org/downloads/t20s_json.zip'
zip1 = zipfile.ZipFile('/content/json_data/main.zip','r')
zip1.extractall('/content/json_data')
zip1.close()

--2022-06-28 13:26:22--  https://cricsheet.org/downloads/t20s_json.zip
Resolving cricsheet.org (cricsheet.org)... 64.90.49.16
Connecting to cricsheet.org (cricsheet.org)|64.90.49.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6504477 (6.2M) [application/zip]
Saving to: ‘/content/json_data/main.zip’

/content/json_data/ 100%[===================>]   6.20M  9.41MB/s    in 0.7s    

2022-06-28 13:26:23 (9.41 MB/s) - ‘/content/json_data/main.zip’ saved [6504477/6504477]



In [3]:
# removing the main ZIP file and a README.txt from the directory
!rm /content/json_data/README.txt
!rm /content/json_data/main.zip

In [5]:
import json
import pandas as pd
import numpy as np

#initializing an empty dataframe
df = pd.DataFrame(columns = ['city','date','venue', 'gender','team_1','team_2','toss_winner','toss_decision','winner','target','margin','victory_margin','MOM','referee','tv_umpire','field_umpire_1','field_umpire_2'])


In [6]:
for j,i in enumerate(os.listdir('/content/json_data')): #looping through all the JSON files in the folder
  g = '/content/json_data/' + i
  data = json.load(open(g))
  try:
    df.loc[j,'city'] = data['info']['city']
  except KeyError:
    df.loc[j,'city'] = np.NaN
  df.loc[j,'date'] = data['info']['dates'][0]
  try:
    df.loc[j,'winner'] = data['info']['outcome']['winner'] 
  except KeyError: 
    try:
      df.loc[j,'winner'] = data['info']['outcome']['eliminator'] #tie-eliminator
    except KeyError:
      df.loc[j,'winner'] = data['info']['outcome']['result'] # no result
  try:
    df.loc[j,'margin'] = str(data['info']['outcome']['by'])
  except KeyError:
    try:
      if data['info']['outcome']['result'] == 'tie':
        df.loc[j,'margin'] = 'Tie-eliminator'
      elif data['info']['outcome']['result'] == 'no result':
        df.loc[j,'margin'] = np.NaN
    except KeyError:
      df.loc[j,'margin'] = data['info']['outcome']['method']
  try:
    df.loc[j,'MOM'] = data['info']['player_of_match'][0]
  except KeyError:
    df.loc[j,'MOM'] = np.NaN
  try:
    df.loc[j,'referee'] = data['info']['officials']['match_referees'][0]
  except KeyError:
    df.loc[j,'referee'] = np.NaN
  try:
    df.loc[j,'tv_umpire'] = data['info']['officials']['tv_umpires'][0]
  except KeyError:
    df.loc[j,'tv_umpire'] = np.NaN
  try:
    df.loc[j,'field_umpire_1'] = data['info']['officials']['umpires'][0]
  except KeyError:
    df.loc[j,'field_umpire_1'] = np.NaN
  try:
    df.loc[j,'field_umpire_2'] = data['info']['officials']['umpires'][1]
  except KeyError:
    df.loc[j,'field_umpire_2'] = np.NaN
  list1 = []
  for key, value in data['info']['players'].items():
    list1.append(key)
  df.loc[j, 'team_1'] = list1[0]
  df.loc[j, 'team_2'] = list1[1]
  df.loc[j, 'gender'] = data['info']['gender']
  df.loc[j, 'toss_winner'] = data['info']['toss']['winner']
  df.loc[j, 'toss_decision'] =  data['info']['toss']['decision']
  df.loc[j, 'venue'] = data['info']['venue']
  try:
    df.loc[j, 'target'] = data['innings'][1]['target']['runs']
  except KeyError:
    df.loc[j, 'target'] = np.NaN
  except IndexError:
    df.loc[j, 'target'] = np.NaN
  data.clear()



Exception reporting mode: Verbose


In [8]:
df['margin'] = df['margin'].str.strip('{}') # transforming dictionaries in margin column to a proper string format
df['margin'] = df['margin'].str.replace(':','') # transforming dictionaries in margin column to a proper string format
df['margin'] = df['margin'].str.replace('\'','') # transforming dictionaries in margin column to a proper string format
df_new = df['margin'].str.split(' ', expand = True) 
df_new[0] = df_new[0].str.replace('\'','')
df['victory_margin'] = df_new[1] + ' ' + df_new[0] # assigning transformed strings to the victory_margin column
df.drop(['margin'],axis = 1, inplace = True) #dropping the redundant margin column


In [10]:
df.head()

,city,date,venue,gender,team_1,team_2,toss_winner,toss_decision,winner,target,victory_margin,MOM,referee,tv_umpire,field_umpire_1,field_umpire_2
0,NaN,2017-01-20,Dubai International Cricket Stadium,male,Afghanistan,Oman,Afghanistan,field,Afghanistan,150,8 wickets,Mohammad Shahzad,GF Labrooy,NaN,AJT Dowdalls,AJ Neill
1,Brighton,2021-07-11,"County Ground, Hove",female,India,England,England,field,India,149,8 runs,DB Sharma,Phil Whitticase,S Redfern,ID Blackwell,PK Baldwin
2,Kuala Lumpur,2019-10-04,Kinrara Academy Oval,male,Malaysia,Vanuatu,Malaysia,bat,Malaysia,207,22 runs,Syed Aziz,Ramesh Menon,NaN,Rudy Ismandy,N Sivan
3,NaN,2017-01-19,Dubai International Cricket Stadium,male,Oman,Scotland,Oman,bat,Scotland,134,7 wickets,MH Cross,GF Labrooy,NaN,Ahmed Shah Pakteen,Iftikhar Ali
4,Barbados,2010-05-06,"Kensington Oval, Bridgetown",male,England,Pakistan,England,field,England,148,6 wickets,KP Pietersen,AG Hurst,SJA Taufel,BF Bowden,RE Koertzen


In [43]:
df_players = pd.DataFrame(columns = ['venue','date','T1','T2']) # A new dataframe to store playing XI details

In [44]:
for j,i in enumerate(os.listdir('/content/json_data')): #looping through all the JSON files in the folder
  player_list = []
  g = '/content/json_data/' + i
  data = json.load(open(g))
  df_players.loc[j, 'venue'] = data['info']['venue']
  df_players.loc[j,'date'] = data['info']['dates'][0]
  for key, value in data['info']['players'].items():
    list1.append(key)
    for player in value:
      player_list.append(player)
  df_players.loc[j,'T1P1'] = player_list[0]
  df_players.loc[j,'T1P2'] = player_list[1]
  df_players.loc[j,'T1P3'] = player_list[2]
  df_players.loc[j,'T1P4'] = player_list[3]
  df_players.loc[j,'T1P5'] = player_list[4]
  df_players.loc[j,'T1P6'] = player_list[5]
  df_players.loc[j,'T1P7'] = player_list[6]
  df_players.loc[j,'T1P8'] = player_list[7]
  df_players.loc[j,'T1P9'] = player_list[8]
  df_players.loc[j,'T1P10'] = player_list[9]
  df_players.loc[j,'T1P11'] = player_list[10]
  df_players.loc[j,'T1P12'] = player_list[11]
  df_players.loc[j,'T1P13'] = player_list[12]
  df_players.loc[j,'T1P14'] = player_list[13]
  df_players.loc[j,'T1P15'] = player_list[14]
  df_players.loc[j,'T1P16'] = player_list[15]
  df_players.loc[j,'T1P17'] = player_list[16]
  df_players.loc[j,'T1P18'] = player_list[17]
  df_players.loc[j,'T1P19'] = player_list[18]
  df_players.loc[j,'T1P20'] = player_list[19]
  df_players.loc[j,'T1P21'] = player_list[20]
  df_players.loc[j,'T1P22'] = player_list[21]  
  df_players.loc[j, 'T1'] = list1[0]
  df_players.loc[j, 'T2'] = list1[1]

In [45]:
df_players.head()

,venue,date,T1,T2,T1P1,T1P2,T1P3,T1P4,T1P5,T1P6,...,T1P13,T1P14,T1P15,T1P16,T1P17,T1P18,T1P19,T1P20,T1P21,T1P22
0,Dubai International Cricket Stadium,2017-01-20,South Africa,Sri Lanka,Mohammad Shahzad,Nawroz Mangal,Asghar Stanikzai,Samiullah Shenwari,Mohammad Nabi,Dawlat Zadran,...,Khawar Ali,Aqib Ilyas,Mehran Khan,Khurram Nawaz,AV Lalcheta,Naseem Khushi,Mohammad Nadeem,Sultan Ahmed,Bilal Khan,Kaleemullah
1,"County Ground, Hove",2021-07-11,South Africa,Sri Lanka,S Mandhana,Shafali Verma,H Kaur,DB Sharma,RM Ghosh,S Rana,...,DN Wyatt,NR Sciver,HC Knight,AE Jones,SIR Dunkley,KH Brunt,S Ecclestone,MK Villiers,S Glenn,FR Davies
2,Kinrara Academy Oval,2019-10-04,South Africa,Sri Lanka,Syed Aziz,Ainool Hafizs,Mohamed Arief,Virandeep Singh,Anwar Arudin,S Muniandy,...,A Mansale,PK Matautaava,N Nipiko,C Tommy,T Langa,J Vira,R Tari,W Nalisa,JW Chilia,ZK Shem
3,Dubai International Cricket Stadium,2017-01-19,South Africa,Sri Lanka,Zeeshan Maqsood,A Poulose,Aqib Ilyas,Khawar Ali,Khurram Nawaz,AV Lalcheta,...,KJ Coetzer,MH Cross,CS MacLeod,CD Wallace,CD de Lange,RD Berrington,SM Sharif,MA Leask,MRJ Watt,CB Sole
4,"Kensington Oval, Bridgetown",2010-05-06,South Africa,Sri Lanka,MJ Lumb,C Kieswetter,KP Pietersen,PD Collingwood,EJG Morgan,LJ Wright,...,Salman Butt,Mohammad Hafeez,Umar Akmal,Shahid Afridi,Misbah-ul-Haq,Abdul Razzaq,Fawad Alam,Mohammad Amir,Saeed Ajmal,Mohammad Asif
